<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Demand-Forecasting-Model-in-Azure-Data-Bricks" data-toc-modified-id="Demand-Forecasting-Model-in-Azure-Data-Bricks-1">Demand Forecasting Model in Azure Data Bricks</a></span></li></ul></div>

## Demand Forecasting Model in Azure Data Bricks 
###### Deepanker Anand, Aviral Mittal
###### Jan 2021

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

Import Python Libraries

In [4]:


 !pip install plotly
 !pip install --upgrade plotly
 !pip install fbprophet

import pandas as pd # Import Pandas for data manipulation using dataframes
import numpy as np # Import Numpy for data statistical analysis 
import matplotlib.pyplot as plt # Import matplotlib for data visualisation
import random
import seaborn as sns
from fbprophet import Prophet
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

import logging
logger = spark._jvm.org.apache.log4j
logging.getLogger("py4j").setLevel(logging.ERROR)

Collecting plotly
 Downloading plotly-4.14.3-py2.py3-none-any.whl (13.2 MB)
 | 10 kB 25.5 MB/s eta 0:00:01 | | 20 kB 19.2 MB/s eta 0:00:01 | | 30 kB 15.7 MB/s eta 0:00:01 | | 40 kB 12.0 MB/s eta 0:00:02 |▏ | 51 kB 11.9 MB/s eta 0:00:02 |▏ | 61 kB 11.8 MB/s eta 0:00:02 |▏ | 71 kB 11.7 MB/s eta 0:00:02 |▏ | 81 kB 11.6 MB/s eta 0:00:02 |▎ | 92 kB 11.5 MB/s eta 0:00:02 |▎ | 102 kB 11.8 MB/s eta 0:00:02 |▎ | 112 kB 11.8 MB/s eta 0:00:02 |▎ | 122 kB 11.8 MB/s eta 0:00:02 |▎ | 133 kB 11.8 MB/s eta 0:00:02 |▍ | 143 kB 11.8 MB/s eta 0:00:02 |▍ | 153 kB 11.8 MB/s eta 0:00:02 |▍ | 163 kB 11.8 MB/s eta 0:00:02 |▍ | 174 kB 11.8 MB/s eta 0:00:02 |▌ | 184 kB 11.8 MB/s eta 0:00:02 |▌ | 194 kB 11.8 MB/s eta 0:00:02 |▌ | 204 kB 11.8 MB/s eta 0:00:02 |▌ | 215 kB 11.8 MB/s eta 0:00:02 |▌ | 225 kB 11.8 MB/s eta 0:00:02 |▋ | 235 kB 11.8 MB/s eta 0:00:02 |▋ | 245 kB 11.8 MB/s eta 0:00:02 |▋ | 256 kB 11.8 MB/s eta 0:00:02 |▋ | 266 kB 11.8 MB/s eta 0:00:02 |▊ | 276 kB 11.8 MB/s eta 0:00:02 |▊ | 286 kB 11.8 MB/s eta 0:00:02 |▊ | 296 kB 11.8 MB/s eta 0:00:02 |▊ | 307 kB 11.8 MB/s eta 0:00:02 |▊ | 317 kB 11.8 MB/s eta 0:00:02 |▉ | 327 kB 11.8 MB/s eta 0:00:02 |▉ | 337 kB 11.8 MB/s eta 0:00:02 |▉ | 348 kB 11.8 MB/s eta 0:00:02 |▉ | 358 kB 11.8 MB/s eta 0:00:02 |█ | 368 kB 11.8 MB/s eta 0:00:02 |█ | 378 kB 11.8 MB/s eta 0:00:02 |█ | 389 kB 11.8 MB/s eta 0:00:02 |█ | 399 kB 11.8 MB/s eta 0:00:02 |█ | 409 kB 11.8 MB/s eta 0:00:02 |█ | 419 kB 11.8 MB/s eta 0:00:02 |█ | 430 kB 11.8 MB/s eta 0:00:02 |█ | 440 kB 11.8 MB/s eta 0:00:02 |█ | 450 kB 11.8 MB/s eta 0:00:02 |█▏ | 460 kB 11.8 MB/s eta 0:00:02 |█▏ | 471 kB 11.8 MB/s eta 0:00:02 |█▏ | 481 kB 11.8 MB/s eta 0:00:02 |█▏ | 491 kB 11.8 MB/s eta 0:00:02 |█▏ | 501 kB 11.8 MB/s eta 0:00:02 |█▎ | 512 kB 11.8 MB/s eta 0:00:02 |█▎ | 522 kB 11.8 MB/s eta 0:00:02 |█▎ | 532 kB 11.8 MB/s eta 0:00:02 |█▎ | 542 kB 11.8 MB/s eta 0:00:02 |█▍ | 552 kB 11.8 MB/s eta 0:00:02 |█▍ | 563 kB 11.8 MB/s eta 0:00:02 |█▍ | 573 kB 11.8 MB/s eta 0:00:02 |█▍ | 583 kB 11.8 MB/s eta 0:00:02 |█▍ | 593 kB 11.8 MB/s eta 0:00:02 |█▌ | 604 kB 11.8 MB/s eta 0:00:02 |█▌ | 614 kB 11.8 MB/s eta 0:00:02 |█▌ | 624 kB 11.8 MB/s eta 0:00:02 |█▌ | 634 kB 11.8 MB/s eta 0:00:02 |█▋ | 645 kB 11.8 MB/s eta 0:00:02 |█▋ | 655 kB 11.8 MB/s eta 0:00:02 |█▋ | 665 kB 11.8 MB/s eta 0:00:02 |█▋ | 675 kB 11.8 MB/s eta 0:00:02 |█▋ | 686 kB 11.8 MB/s eta 0:00:02 |█▊ | 696 kB 11.8 MB/s eta 0:00:02 |█▊ | 706 kB 11.8 MB/s eta 0:00:02 |█▊ | 716 kB 11.8 MB/s eta 0:00:02 |█▊ | 727 kB 11.8 MB/s eta 0:00:02 |█▉ | 737 kB 11.8 MB/s eta 0:00:02 |█▉ | 747 kB 11.8 MB/s eta 0:00:02 |█▉ | 757 kB 11.8 MB/s eta 0:00:02 |█▉ | 768 kB 11.8 MB/s eta 0:00:02 |█▉ | 778 kB 11.8 MB/s eta 0:00:02 |██ | 788 kB 11.8 MB/s eta 0:00:02 |██ | 798 kB 11.8 MB/s eta 0:00:02 |██ | 808 kB 11.8 MB/s eta 0:00:02 |██ | 819 kB 11.8 MB/s eta 0:00:02 |██ | 829 kB 11.8 MB/s eta 0:00:02 |██ | 839 kB 11.8 MB/s eta 0:00:02 |██ | 849 kB 11.8 MB/s eta 0:00:02 |██ | 860 kB 11.8 MB/s eta 0:00:02 |██▏ | 870 kB 11.8 MB/s eta 0:00:02 |██▏ | 880 kB 11.8 MB/s eta 0:00:02 |██▏ | 890 kB 11.8 MB/s eta 0:00:02 |██▏ | 901 kB 11.8 MB/s eta 0:00:02 |██▏ | 911 kB 11.8 MB/s eta 0:00:02 |██▎ | 921 kB 11.8 MB/s eta 0:00:02 |██▎ | 931 kB 11.8 MB/s eta 0:00:02 |██▎ | 942 kB 11.8 MB/s eta 0:00:02 |██▎ | 952 kB 11.8 MB/s eta 0:00:02 |██▍ | 962 kB 11.8 MB/s eta 0:00:02 |██▍ | 972 kB 11.8 MB/s eta 0:00:02 |██▍ | 983 kB 11.8 MB/s eta 0:00:02 |██▍ | 993 kB 11.8 MB/s eta 0:00:02 |██▍ | 1.0 MB 11.8 MB/s eta 0:00:02 |██▌ | 1.0 MB 11.8 MB/s eta 0:00:02 |██▌ | 1.0 MB 11.8 MB/s eta 0:00:02 |██▌ | 1.0 MB 11.8 MB/s eta 0:00:02 |██▌ | 1.0 MB 11.8 MB/s eta 0:00:02 |██▋ | 1.1 MB 11.8 MB/s eta 0:00:02 |██▋ | 1.1 MB 11.8 MB/s eta 0:00:02 |██▋ | 1.1 MB 11.8 MB/s eta 0:00:02 |██▋ | 1.1 MB 11.8 MB/s eta 0:00:02 |██▋ | 1.1 MB 11.8 MB/s eta 0:00:02 |██▊ | 1.1 MB 11.8 MB/s eta 0:00:02 |██▊ | 1.1 MB 11.8 MB/s eta 0:00:02 |██▊ | 1.1 MB 11.8 MB/s eta 0:0

In [5]:
!pip install plotly



Requirement already satisfied: plotly in /databricks/python3/lib/python3.7/site-packages (4.14.3)
Requirement already satisfied: six in /databricks/python3/lib/python3.7/site-packages (from plotly) (1.14.0)
Requirement already satisfied: retrying>=1.3.3 in /databricks/python3/lib/python3.7/site-packages (from plotly) (1.3.3)
 WARNING: You are using pip version 20.0.2; however, version 20.3.3 is available.
You should consider upgrading via the '/databricks/python3/bin/python3.7 -m pip install --upgrade pip' command.

In [6]:
!pip install cufflinks --upgrade
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

Collecting cufflinks
 Downloading cufflinks-0.17.3.tar.gz (81 kB)
████ | 10 kB 17.0 MB/s eta 0:00:01 |████████ | 20 kB 19.4 MB/s eta 0:00:01 |████████████ | 30 kB 13.6 MB/s eta 0:00:01 |████████████████ | 40 kB 7.2 MB/s eta 0:00:01 |████████████████████ | 51 kB 8.4 MB/s eta 0:00:01 |████████████████████████ | 61 kB 8.2 MB/s eta 0:00:01 |████████████████████████████ | 71 kB 8.5 MB/s eta 0:00:01 |████████████████████████████████| 81 kB 4.3 MB/s 
Requirement already satisfied, skipping upgrade: numpy>=1.9.2 in /databricks/python3/lib/python3.7/site-packages (from cufflinks) (1.18.1)
Requirement already satisfied, skipping upgrade: pandas>=0.19.2 in /databricks/python3/lib/python3.7/site-packages (from cufflinks) (1.2.0)
Requirement already satisfied, skipping upgrade: plotly>=4.1.1 in /databricks/python3/lib/python3.7/site-packages (from cufflinks) (4.14.3)
Requirement already satisfied, skipping upgrade: six>=1.9.0 in /databricks/python3/lib/python3.7/site-packages (from cufflinks) (1.14.0)
Collecting colorlover>=0.2.1
 Downloading colorlover-0.3.0-py3-none-any.whl (8.9 kB)
Requirement already satisfied, skipping upgrade: setuptools>=34.4.1 in /usr/local/lib/python3.7/dist-packages (from cufflinks) (45.2.0)
Requirement already satisfied, skipping upgrade: ipython>=5.3.0 in /databricks/python3/lib/python3.7/site-packages (from cufflinks) (7.12.0)
Collecting ipywidgets>=7.0.0
 Downloading ipywidgets-7.6.3-py2.py3-none-any.whl (121 kB)
██▊ | 10 kB 16.0 MB/s eta 0:00:01 |█████▍ | 20 kB 13.2 MB/s eta 0:00:01 |████████ | 30 kB 9.3 MB/s eta 0:00:01 |██████████▊ | 40 kB 8.1 MB/s eta 0:00:01 |█████████████▌ | 51 kB 7.6 MB/s eta 0:00:01 |████████████████▏ | 61 kB 7.4 MB/s eta 0:00:01 |██████████████████▉ | 71 kB 7.8 MB/s eta 0:00:01 |█████████████████████▌ | 81 kB 7.5 MB/s eta 0:00:01 |████████████████████████▏ | 92 kB 7.8 MB/s eta 0:00:01 |███████████████████████████ | 102 kB 7.6 MB/s eta 0:00:01 |█████████████████████████████▋ | 112 kB 7.6 MB/s eta 0:00:01 |████████████████████████████████| 121 kB 7.6 MB/s 
Requirement already satisfied, skipping upgrade: python-dateutil>=2.7.3 in /databricks/python3/lib/python3.7/site-packages (from pandas>=0.19.2->cufflinks) (2.8.1)
Requirement already satisfied, skipping upgrade: pytz>=2017.3 in /databricks/python3/lib/python3.7/site-packages (from pandas>=0.19.2->cufflinks) (2019.3)
Requirement already satisfied, skipping upgrade: retrying>=1.3.3 in /databricks/python3/lib/python3.7/site-packages (from plotly>=4.1.1->cufflinks) (1.3.3)
Requirement already satisfied, skipping upgrade: prompt-toolkit!=3.0.0,!=3.0.1,<3.1.0,>=2.0.0 in /databricks/python3/lib/python3.7/site-packages (from ipython>=5.3.0->cufflinks) (3.0.3)
Requirement already satisfied, skipping upgrade: pexpect; sys_platform != "win32" in /databricks/python3/lib/python3.7/site-packages (from ipython>=5.3.0->cufflinks) (4.8.0)
Requirement already satisfied, skipping upgrade: decorator in /databricks/python3/lib/python3.7/site-packages (from ipython>=5.3.0->cufflinks) (4.4.1)
Requirement already satisfied, skipping upgrade: jedi>=0.10 in /databricks/python3/lib/python3.7/site-packages (from ipython>=5.3.0->cufflinks) (0.17.2)
Requirement already satisfied, skipping upgrade: pygments in /databricks/python3/lib/python3.7/site-packages (from ipython>=5.3.0->cufflinks) (2.5.2)
Requirement already satisfied, skipping upgrade: pickleshare in /databricks/python3/lib/python3.7/site-packages (from ipython>=5.3.0->cufflinks) (0.7.5)
Requirement already satisfied, skipping upgrade: traitlets>=4.2 in /databricks/python3/lib/python3.7/site-packages (from ipython>=5.3.0->cufflinks) (4.3.3)
Requirement already satisfied, skipping upgrade: backcall in /databricks/python3/lib/python3.7/site-packages (from ipython>=5.3.0->cufflinks) (0.1.0)
Requirement already satisfied, skipping upgrade: ipykernel>=4.5.1 in /databricks/python3/lib/python3.7/site-packages (from ipywidgets>=7.0.0->cufflinks) (5.1.4)
Collecting wid

In [7]:
import os
print(os.getcwd())
print(os.path.abspath('../tables'))

# Change working directory
os.chdir('/dbfs/FileStore/shared_uploads/deepanker.anand@absolutdata.com/')
print(os.getcwd())

In [8]:
# File location and type
file_location = "/FileStore/tables/train_demand_forecasting.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

display(df)

In [9]:
# Create a view or table

temp_table_name = "train_demand_forecasting_csv"

df.createOrReplaceTempView(temp_table_name)

In [10]:
%sql

/* Query the created temp table in a SQL cell */

select * from `train_demand_forecasting_csv`

In [11]:
# With this registered as a temp view, it will only be available to this particular notebook. If you'd like other users to be able to query this table, you can also create a table from the DataFrame.

# Once saved, this table will persist across cluster restarts as well as allow various users across different notebooks to query this data.

# To do so, choose your table name and uncomment the bottom line.

permanent_table_name = "train_demand_forecasting_csv"

# df.write.format("parquet").saveAsTable(permanent_table_name)

In [12]:
# Make connection to the Azure Blob Storage

storage_account_name = "deepankerstorage1"
storage_account_access_key = "GScZye6mFi/5Jx884+6GTpOOhy16ka8jtj/Y5not/ZKmhUyq6CJQFSMHc05i/uFgRhzJik7cvlpy8Q5eToyfhQ=="
blobContainerName="datasets"

In [13]:
file_location = "wasbs://deepankerstorage1.blob.core.windows.net/mycontainer/demand_forecasting/train_demand_forecasting.csv".format(blobContainerName, storage_account_name)
file_type = "csv"

# https://deepankerstorage1.blob.core.windows.net/train_demand_forecasting.csv

# wasbs://{}@{}.blob.core.windows.net/mycontainer/demand_forecasting/train_demand_forecasting.csv

In [14]:
spark.conf.set(
  "fs.azure.account.key."+storage_account_name+".blob.core.windows.net",
  storage_account_access_key) #spark context 

In [15]:
### Commenting this code as it throws error (RERUN THIS ONCE THIS ISSUE GETS RESOLVED)

# df = spark.read.format(file_type).option("inferSchema", "true").load(file_location)

In [16]:
 # load fbprophet library
# dbutils.library.installPyPI('FBProphet', version='0.5') # find latest version of fbprophet here: https://pypi.org/project/fbprophet/
# dbutils.library.installPyPI('holidays','0.9.12') # this line is in response to this issue with fbprophet 0.5: 
# https://github.com/facebook/prophet/issues/1293

# dbutils.library.restartPython()

In [17]:
from pyspark.sql.types import *

# structure of the training data set
train_schema = StructType([
  StructField('date', DateType()),
  StructField('store', IntegerType()),
  StructField('item', IntegerType()),
  StructField('sales', IntegerType())
  ])

# read the training file into a dataframe
train = spark.read.csv(
  '/FileStore/tables/train_demand_forecasting.csv', 
  header=True, 
  schema=train_schema
  )

# make the dataframe queriable as a temporary view
train.createOrReplaceTempView('train')

In [18]:
train.show()

In [19]:
train.printSchema()

In [20]:
display(train)

In [21]:
# convert pyspark dataframe to pandas dataframe
train=train.toPandas()
type(train)

Read raw datasets from DBFS location

In [25]:
# Reading file from DBFS Storage 
import pandas as pd

# df1 = pd.read_csv("/dbfs/FileStore/shared_uploads/deepanker.anand@absolutdata.com/sample_submission.csv")
# df2 = pd.read_csv("/dbfs/FileStore/shared_uploads/deepanker.anand@absolutdata.com/test_demand_forecasting.csv")
#df3 = pd.read_csv("/dbfs/FileStore/shared_uploads/deepanker.anand@absolutdata.com/train_demand_forecasting.csv")
df3 = pd.read_csv("/dbfs/FileStore/shared_uploads/aviral.mittal@absolutdata.com/train-1.csv")

In [26]:
train=df3
train.head()

Out[3]:

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10


In [27]:
train.shape

In [28]:
train.info()

In [29]:
train.columns

In [30]:
# count no of missing values in entire dataframe 
train.isnull().sum().sum()

In [31]:
# count no of missing values in each column of dataframe 
train.isnull().sum()

In [32]:
# Descriptive statistics   
train.describe(percentiles=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0])

In [33]:
train.dtypes

In [34]:
# train=train.sort_values(['date'])

In [35]:
print(len(train)) # Count total cells for each column  
print(train.count()) # Count non-NA cells for each column 

In [36]:
train['store'].unique().tolist()  

In [37]:
train['store'].value_counts(normalize=True, sort=True, ascending=False) 

In [38]:
import pandas as pd

# Converting sales to nummeric column
train["sales"]=train["sales"].astype(int)
train["year"] = pd.to_datetime(train["date"]).dt.year
train["month"] = pd.to_datetime(train["date"]).dt.month
train['weekday'] = pd.to_datetime(train["date"]).dt.dayofweek
train['quarter'] = pd.to_datetime(train["date"]).dt.quarter
train['dayofmonth'] = pd.to_datetime(train["date"]).dt.day
train['dayofyear'] = pd.to_datetime(train["date"]).dt.dayofyear
train["month_year"] = pd.to_datetime(train["year"] * 10000 + train["month"] * 100 + 1, format="%Y%m%d")


train.tail()

Out[4]:

,date,store,item,sales,year,month,weekday,quarter,dayofmonth,dayofyear,month_year
912995,2017-12-27,10,50,63,2017,12,2,4,27,361,2017-12-01
912996,2017-12-28,10,50,59,2017,12,3,4,28,362,2017-12-01
912997,2017-12-29,10,50,74,2017,12,4,4,29,363,2017-12-01
912998,2017-12-30,10,50,62,2017,12,5,4,30,364,2017-12-01
912999,2017-12-31,10,50,82,2017,12,6,4,31,365,2017-12-01


In [39]:
# checking for missing values 
count_nan = len(train) - train.count()
count_nan

train.dtypes

In [40]:
# drop any missing records
train = train.dropna()

In [41]:
train.info()

Perform Exploratory Data Analysis (EDA) on raw dataset

* Looking at time perspective 
* Analyse from Item perspective
* Analyse from Store perspective

In [44]:
import matplotlib.pyplot as plt

plt.hist(train["sales"])
plt.show()

In [45]:
import seaborn as sns

mean_sales = train.groupby(["date"], as_index=False)
mean_sales = mean_sales[["sales"]].sum()
#mean_sales["idx"] = mean_sales.index
g = sns.relplot(x="date", y="sales", data=mean_sales, kind="line")

In [46]:
# mean sales for each item 
sales_vs_item=train[['item','sales']]
sales_vs_item.set_index('item',inplace=True)
sales_vs_item

data=sales_vs_item.groupby(['item'], as_index=True).mean()
data
sns.distplot(data['sales'])

In [47]:
#import plotly as py 
# Time Series Analysis for Store 10 and Item 40
# performing time series analysis for particular item of store
print('Before Filtering '+str(train.shape))

# let consider particular item and store
# store=10
# item=40

sample=train[train.store==10]
sample=sample[sample.item==40]
print('After Filtering '+str(sample.shape))

sns.lineplot(x="date", y="sales", data=sample)

In [48]:
%sql  --annual trend in unit sales

SELECT
  year(date) as year, 
  sum(sales) as sales
FROM train
GROUP BY 1
ORDER BY 1; 

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: org.apache.spark.sql.AnalysisException: Table or view not found: train; line 4 pos 5;
'Sort [unresolvedordinal(1) ASC NULLS FIRST], true
+- 'Aggregate [unresolvedordinal(1)], ['year('date) AS year#45, 'sum('sales) AS sales#46]
 +- 'UnresolvedRelation [train]

	at org.apache.spark.sql.catalyst.analysis.package$AnalysisErrorAt.failAnalysis(package.scala:42)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis$1(CheckAnalysis.scala:113)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.$anonfun$checkAnalysis$1$adapted(CheckAnalysis.scala:95)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:190)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1(TreeNode.scala:189)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1$adapted(TreeNode.scala:189)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:189)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1(TreeNode.scala:189)
	at org.apache.spark.sql.catalyst.trees.TreeNode.$anonfun$foreachUp$1$adapted(TreeNode.scala:189)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at org.apache.spark.sql.catalyst.trees.TreeNode.foreachUp(TreeNode.scala:189)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.checkAnalysis(CheckAnalysis.scala:95)
	at org.apache.spark.sql.catalyst.analysis.CheckAnalysis.checkAnalysis$(CheckAnalysis.scala:92)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.checkAnalysis(Analyzer.scala:181)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.$anonfun$executeAndCheck$1(Analyzer.scala:204)
	at org.apache.spark.sql.catalyst.plans.logical.AnalysisHelper$.markInAnalyzer(AnalysisHelper.scala:223)
	at org.apache.spark.sql.catalyst.analysis.Analyzer.executeAndCheck(Analyzer.scala:201)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$analyzed$1(QueryExecution.scala:91)
	at org.apache.spark.sql.catalyst.QueryPlanningTracker.measurePhase(QueryPlanningTracker.scala:136)
	at org.apache.spark.sql.execution.QueryExecution.$anonfun$executePhase$1(QueryExecution.scala:171)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:841)
	at org.apache.spark.sql.execution.QueryExecution.executePhase(QueryExecution.scala:171)
	at org.apache.spark.sql.execution.QueryExecution.analyzed$lzycompute(QueryExecution.scala:92)
	at org.apache.spark.sql.execution.QueryExecution.analyzed(QueryExecution.scala:89)
	at org.apache.spark.sql.execution.QueryExecution.assertAnalyzed(QueryExecution.scala:81)
	at org.apache.spark.sql.Dataset$.$anonfun$ofRows$2(Dataset.scala:103)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:841)
	at org.apache.spark.sql.Dataset$.ofRows(Dataset.scala:101)
	at org.apache.spark.sql.SparkSession.$anonfun$sql$1(SparkSession.scala:676)
	at org.apache.spark.sql.SparkSession.withActive(SparkSession.scala:841)
	at org.apache.spark.sql.SparkSession.sql(SparkSession.scala:671)
	at org.apache.spark.sql.SQLContext.sql(SQLContext.scala:672)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.$anonfun$executeSql$1(SQLDriverLocal.scala:91)
	at scala.collection.TraversableLike.$anonfun$map$1(TraversableLike.scala:238)
	at scala.collection.immutable.List.foreach(List.scala:392)
	at scala.collection.TraversableLike.map(TraversableLike.scala:238)
	at scala.collection.TraversableLike.map$(TraversableLike.scala:231)
	at scala.collection.immutable.List.map(List.scala:298)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.executeSql(SQLDriverLocal.scala:37)
	at com.databricks.backend.daemon.driver.SQLDriverLocal.repl(SQLDriverLocal.scala:144)
	at com.databricks.backend.daemon.driver.DriverLocal.$anonfun$execute$11(DriverLocal.scala:451)
	at com.databricks.logging.UsageLogging.$anonfun$withAttributionContext$1(UsageLogging.scala:240)
	at scala.util.DynamicVar

In [49]:
%sql  -- If we aggregate the data around the individual months in each year, a distinct yearly seasonal pattern is observed which seems to grow in # scale with overall growth in sales

SELECT 
  TRUNC(date, 'MM') as month,
  SUM(sales) as sales
FROM train
GROUP BY 1
ORDER BY 1;

In [50]:
train['weekday'].value_counts()

In [51]:
%sql

--drop table x;

--create table x as
SELECT 
    date,
    SUM(sales) as sales
  FROM train
  GROUP BY 1;
  
--select * from x limit 10;  

In [52]:
%sql -- Aggregating the data at a weekday level, a pronounced weekly seasonal pattern is observed with a peak on Sunday (weekday 0), a hard drop on Monday (weekday 1) and then a steady pickup over the week heading back to the Sunday high. This pattern seems to be pretty stable across the five years of observations:

-- SELECT YEAR(date) as year,
--   CAST(DATE_FORMAT(date, 'u') as Integer) % 7 as weekday,
--   avg(sales) as sales
-- FROM (
--   SELECT 
--     date,
--     SUM(sales) as sales
--   FROM train
--   GROUP BY date
--      ) x
-- GROUP BY 1, CAST(DATE_FORMAT(date, 'u') as Integer)
-- ORDER BY 1, 2;

In [53]:
sales_vs_item=train[['item','sales']]
sales_vs_item.set_index('item',inplace=True)


plt.figure(figsize=(10,5))
sns.distplot(sales_vs_item['sales'])

# Some items has high demand in comparision to other

In [54]:
# Avg. sales for each item 
data=sales_vs_item.groupby(['item']).mean()
plt.figure(figsize=(20,8))
sns.barplot(data.index,data['sales'])

In [55]:
# Sales trend over the months and year

# train['Month']=train['date'].dt.month
# train['Year']=train['date'].dt.year
plt.figure(figsize=(10,30))
sns.catplot(data = train, x ='month', y = "sales",row = "year")
plt.show()

In [56]:
# Understand sales data distribution across the stores
import numpy as np
store_df = train.copy()
sales_pivoted_df = pd.pivot_table(store_df, index='store', values=['sales'], columns='item', aggfunc=np.mean)
# Pivoted dataframe
display(sales_pivoted_df)

# 10 rows × 50 columns

In [57]:
store_data=train[['store','sales']]
store_data.set_index('store',inplace=True)
store_data=store_data.groupby('store').mean()
store_data['store'] = store_data.index
store_data.head()

In [58]:
plt.figure(figsize=(15,5))
store_data.sort_values('sales')
sns.barplot(x=store_data.index,y=store_data['sales'],order=store_data.sort_values('sales').store) # Bars are ordered in ascending order  

Build a Forecasting Model

In [60]:
train_data=train.rename(columns={'date':'ds','sales':'y'})
train_data.head(5)

Out[5]:

,ds,store,item,y,year,month,weekday,quarter,dayofmonth,dayofyear,month_year
0,2013-01-01,1,1,13,2013,1,1,1,1,1,2013-01-01
1,2013-01-02,1,1,11,2013,1,2,1,2,2,2013-01-01
2,2013-01-03,1,1,14,2013,1,3,1,3,3,2013-01-01
3,2013-01-04,1,1,13,2013,1,4,1,4,4,2013-01-01
4,2013-01-05,1,1,10,2013,1,5,1,5,5,2013-01-01


In [61]:
 !pip install fbprophet
 !pip install --upgrade plotly 

Collecting fbprophet
 Downloading fbprophet-0.7.1.tar.gz (64 kB)
█████▏ | 10 kB 20.3 MB/s eta 0:00:01 |██████████▎ | 20 kB 22.3 MB/s eta 0:00:01 |███████████████▍ | 30 kB 17.4 MB/s eta 0:00:01 |████████████████████▌ | 40 kB 7.2 MB/s eta 0:00:01 |█████████████████████████▋ | 51 kB 8.6 MB/s eta 0:00:01 |██████████████████████████████▊ | 61 kB 7.0 MB/s eta 0:00:01 |████████████████████████████████| 64 kB 1.9 MB/s 
Requirement already satisfied: Cython>=0.22 in /databricks/python3/lib/python3.7/site-packages (from fbprophet) (0.29.15)
Collecting cmdstanpy==0.9.5
 Downloading cmdstanpy-0.9.5-py3-none-any.whl (37 kB)
Collecting pystan>=2.14
 Downloading pystan-2.19.1.1-cp37-cp37m-manylinux1_x86_64.whl (67.3 MB)
 | 10 kB 29.3 MB/s eta 0:00:03 | | 20 kB 19.4 MB/s eta 0:00:04 | | 30 kB 14.5 MB/s eta 0:00:05 | | 40 kB 13.7 MB/s eta 0:00:05 | | 51 kB 12.8 MB/s eta 0:00:06 | | 61 kB 12.2 MB/s eta 0:00:06 | | 71 kB 12.2 MB/s eta 0:00:06 | | 81 kB 11.7 MB/s eta 0:00:06 | | 92 kB 11.5 MB/s eta 0:00:06 | | 102 kB 11.2 MB/s eta 0:00:06 | | 112 kB 11.2 MB/s eta 0:00:06 | | 122 kB 11.2 MB/s eta 0:00:06 | | 133 kB 11.2 MB/s eta 0:00:06 | | 143 kB 11.2 MB/s eta 0:00:06 | | 153 kB 11.2 MB/s eta 0:00:06 | | 163 kB 11.2 MB/s eta 0:00:06 | | 174 kB 11.2 MB/s eta 0:00:06 | | 184 kB 11.2 MB/s eta 0:00:06 | | 194 kB 11.2 MB/s eta 0:00:06 | | 204 kB 11.2 MB/s eta 0:00:06 | | 215 kB 11.2 MB/s eta 0:00:06 | | 225 kB 11.2 MB/s eta 0:00:06 |▏ | 235 kB 11.2 MB/s eta 0:00:06 |▏ | 245 kB 11.2 MB/s eta 0:00:06 |▏ | 256 kB 11.2 MB/s eta 0:00:06 |▏ | 266 kB 11.2 MB/s eta 0:00:06 |▏ | 276 kB 11.2 MB/s eta 0:00:06 |▏ | 286 kB 11.2 MB/s eta 0:00:06 |▏ | 296 kB 11.2 MB/s eta 0:00:06 |▏ | 307 kB 11.2 MB/s eta 0:00:06 |▏ | 317 kB 11.2 MB/s eta 0:00:06 |▏ | 327 kB 11.2 MB/s eta 0:00:06 |▏ | 337 kB 11.2 MB/s eta 0:00:06 |▏ | 348 kB 11.2 MB/s eta 0:00:06 |▏ | 358 kB 11.2 MB/s eta 0:00:06 |▏ | 368 kB 11.2 MB/s eta 0:00:06 |▏ | 378 kB 11.2 MB/s eta 0:00:06 |▏ | 389 kB 11.2 MB/s eta 0:00:06 |▏ | 399 kB 11.2 MB/s eta 0:00:06 |▏ | 409 kB 11.2 MB/s eta 0:00:06 |▏ | 419 kB 11.2 MB/s eta 0:00:06 |▏ | 430 kB 11.2 MB/s eta 0:00:06 |▏ | 440 kB 11.2 MB/s eta 0:00:06 |▏ | 450 kB 11.2 MB/s eta 0:00:06 |▏ | 460 kB 11.2 MB/s eta 0:00:06 |▎ | 471 kB 11.2 MB/s eta 0:00:06 |▎ | 481 kB 11.2 MB/s eta 0:00:06 |▎ | 491 kB 11.2 MB/s eta 0:00:06 |▎ | 501 kB 11.2 MB/s eta 0:00:06 |▎ | 512 kB 11.2 MB/s eta 0:00:06 |▎ | 522 kB 11.2 MB/s eta 0:00:06 |▎ | 532 kB 11.2 MB/s eta 0:00:06 |▎ | 542 kB 11.2 MB/s eta 0:00:06 |▎ | 552 kB 11.2 MB/s eta 0:00:06 |▎ | 563 kB 11.2 MB/s eta 0:00:06 |▎ | 573 kB 11.2 MB/s eta 0:00:06 |▎ | 583 kB 11.2 MB/s eta 0:00:06 |▎ | 593 kB 11.2 MB/s eta 0:00:06 |▎ | 604 kB 11.2 MB/s eta 0:00:06 |▎ | 614 kB 11.2 MB/s eta 0:00:06 |▎ | 624 kB 11.2 MB/s eta 0:00:06 |▎ | 634 kB 11.2 MB/s eta 0:00:06 |▎ | 645 kB 11.2 MB/s eta 0:00:06 |▎ | 655 kB 11.2 MB/s eta 0:00:06 |▎ | 665 kB 11.2 MB/s eta 0:00:06 |▎ | 675 kB 11.2 MB/s eta 0:00:06 |▎ | 686 kB 11.2 MB/s eta 0:00:06 |▎ | 696 kB 11.2 MB/s eta 0:00:06 |▍ | 706 kB 11.2 MB/s eta 0:00:06 |▍ | 716 kB 11.2 MB/s eta 0:00:06 |▍ | 727 kB 11.2 MB/s eta 0:00:06 |▍ | 737 kB 11.2 MB/s eta 0:00:06 |▍ | 747 kB 11.2 MB/s eta 0:00:06 |▍ | 757 kB 11.2 MB/s eta 0:00:06 |▍ | 768 kB 11.2 MB/s eta 0:00:06 |▍ | 778 kB 11.2 MB/s eta 0:00:06 |▍ | 788 kB 11.2 MB/s eta 0:00:06 |▍ | 798 kB 11.2 MB/s eta 0:00:06 |▍ | 808 kB 11.2 MB/s eta 0:00:06 |▍ | 819 kB 11.2 MB/s eta 0:00:06 |▍ | 829 kB 11.2 MB/s eta 0:00:06 |▍ | 839 kB 11.2 MB/s eta 0:00:06 |▍ | 849 kB 11.2 MB/s eta 0:00:06 |▍ | 860 kB 11.2 MB/s eta 0:00:06 |▍ | 870 kB 11.2 MB/s eta 0:00:06 |▍ | 880 kB 11.2 MB/s eta 0:00:06 |▍ | 890 kB 11.2 MB/s eta 0:00:06 |▍ | 901 kB 11.2 MB/s eta 0:00:06 |▍ | 911 kB 11.2 MB/s eta 0:00:06 |▍ | 921 kB 11.2 MB/s eta 0:00:06 |▍ | 931 kB 11.2 MB/s eta 0:00:06 |▌ | 942 kB 11.2 MB/s eta 0:00:06 |▌ | 952 kB 11.2 MB/s eta 0:00:06 |▌ | 962 kB 11.2 MB/s eta 0:00:06 |▌ | 972 kB 1

In [62]:
from fbprophet import Prophet
import plotly.offline as py
import logging

# disable informational messages from fbprophet
logging.getLogger('py4j').setLevel(logging.ERROR)

In [63]:
# Dataframe to be used for all the algorithms
filtered=train_data[(train_data['item']==50) & (train_data['store']==10)]
traindf=filtered[filtered["year"]<2017]
testdf=filtered[filtered['year']==2017]

traindf_ts=traindf[['month_year','y']]
traindf_ts=traindf_ts.groupby("month_year").sum()

testdf_ts=testdf[['month_year','y']]
testdf_ts=testdf_ts.groupby("month_year").sum()



In [64]:

filtered_train=filtered[filtered['year']<2017] 


filtered_train=filtered_train[['ds','y']]

# set model parameters
m = Prophet(
  interval_width=0.95,
  growth='linear',
  daily_seasonality=False,
  weekly_seasonality=True,
  yearly_seasonality=True,
  seasonality_mode='multiplicative'
  )

# fit the model to historical data
m.fit(filtered_train)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-3018600603569583> in <module> 
 5 
 6 # set model parameters 
 ----> 7 m = Prophet(
 8 interval_width = 0.95 , 
 9 growth = 'linear' , 

 NameError : name 'Prophet' is not defined

In [65]:
# Define a dataset including both historical dates & 365-days beyond the last available date
future_pd = m.make_future_dataframe(
  periods=365, 
  freq='d', 
  include_history=True
  )

future_pd.tail()

In [66]:
# predict over the dataset
forecast_pd = m.predict(future_pd)
forecast_pd[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

In [67]:
# Check the general and seasonal trends in our forecast model 
trends_fig = m.plot_components(forecast_pd)
trends_fig

In [68]:
predict_fig = m.plot( forecast_pd, xlabel='date', ylabel='sales')
display(predict_fig)

The black dots represent our actuals with the darker blue line representing our predictions and the lighter blue band representing our (95%) uncertainty interval.

In [70]:
# Check how our actual and predicted data line up as well as a forecast for the future

predict_fig = m.plot( forecast_pd, xlabel='date', ylabel='sales')

# adjust figure to display dates from last year + the 365 day forecast
xlim = predict_fig.axes[0].get_xlim()
new_xlim = ( xlim[1]-(180.0+365.0), xlim[1]-365.0)
predict_fig.axes[0].set_xlim(new_xlim)

display(predict_fig)

In [71]:
filtered['ds'] = filtered['ds'].astype('datetime64[ns]')

In [72]:
# Calculate Model Performance Metrics to evaulate the forecast model 
# Mean Absolute Error, Mean Squared Error and Root Mean Squared Error values for the predicted relative to the actual values in our dataset

from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt
from datetime import date

import pandas as pd


# get historical actuals & predictions for comparison
actuals_pd = filtered_train[ pd.to_datetime(filtered_train['ds']) < pd.to_datetime(date(2017, 1, 1)) ]['y']
predicted_pd = forecast_pd[ pd.to_datetime(forecast_pd['ds']) < pd.to_datetime(date(2017, 1, 1)) ]['yhat']

# calculate evaluation metrics
mae = mean_absolute_error(actuals_pd, predicted_pd)
mse = mean_squared_error(actuals_pd, predicted_pd)
rmse = sqrt(mse)

# print metrics to the screen
print( '\n'.join(['MAE: {0}', 'MSE: {1}', 'RMSE: {2}']).format(mae, mse, rmse) )

In [73]:
filtered.head() # completed data for item=50 and store=10 from 2013-2017

In [74]:
filtered_future=filtered[filtered['year']==2017] # filter the forecast period from complete dataset

a=filtered_future[['ds','y']] # actual
b=forecast_pd[['ds','yhat', 'yhat_lower', 'yhat_upper']] # predicted

# set index as ds in both datasets 
filtered.set_index('ds')
b.set_index('ds')

#filtered['ds']=filtered['ds'].astype('datetime64[ns]')
#b['ds']=b['ds'].astype('datetime64[ns]')
#final=filtered.join(b, on="ds")

filtered['ds'] = filtered['ds'].astype('datetime64[ns]')
filtered.set_index('ds')
b.set_index('ds')
final=filtered.merge(b, how='inner')


#a['ds'] = pd.to_datetime(a['ds'],  format='%Y%m%d')
#a['ds'] =  pd.to_datetime(a['ds'])
#a.join(b, on='ds')

#pd.concat([a,b])
#pd.concat(left=a, right=b, how="inner")
#a['ds']=a['ds']

In [75]:
final.head()

In [76]:
final.shape

In [77]:
# Export forecast model results into CSV file
final.to_csv('/dbfs/FileStore/shared_uploads/deepanker.anand@absolutdata.com/fbp_forecast_1.csv')

In [78]:
# MAPE 
import numpy as np

#final.to_csv('/dbfs/FileStore/fbp_forecast_1.csv')

y_actual=final['y']
y_pred=final['yhat']
MAPE=np.mean(np.abs((y_actual - y_pred) / y_actual)) * 100
print(MAPE)

--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-3018600603569596> in <module> 
 4 #final.to_csv('/dbfs/FileStore/fbp_forecast_1.csv') 
 5 
 ----> 6 y_actual = final [ 'y' ] 
 7 y_pred = final [ 'yhat' ] 
 8 MAPE = np . mean ( np . abs ( ( y_actual - y_pred ) / y_actual ) ) * 100 

 NameError : name 'final' is not defined

In [79]:

filtered.head()

In [80]:
import matplotlib.pyplot as plt
import numpy as np

filtered_train_arima=filtered[filtered["year"]<2017]
filtered_test_arima=filtered[filtered["year"]>=2017]

filtered_train_arima.tail()

filtered_ts=filtered_train_arima[["month_year", "y"]]
filtered_ts=filtered_ts.groupby("month_year").sum()
filtered_ts_log=filtered_ts.copy()
filtered_ts_log['y']=np.log(filtered_ts_log['y'])


filtered_test_ts=filtered_test_arima[["month_year", "y"]]
filtered_test_ts=filtered_test_ts.groupby("month_year").sum()

filtered_test_ts

Out[23]:

,y
month_year,
2017-01-01,1744
2017-02-01,1733
2017-03-01,2281
2017-04-01,2721
2017-05-01,2900
2017-06-01,3049
2017-07-01,3311
2017-08-01,2867
2017-09-01,2586


In [81]:
!pip install pmdarima
#from pyramid.arima import auto_arima
from pmdarima import auto_arima


Requirement already satisfied: pmdarima in /databricks/python3/lib/python3.7/site-packages (1.8.0)
Requirement already satisfied: joblib>=0.11 in /databricks/python3/lib/python3.7/site-packages (from pmdarima) (0.14.1)
Requirement already satisfied: urllib3 in /databricks/python3/lib/python3.7/site-packages (from pmdarima) (1.25.8)
Requirement already satisfied: statsmodels!=0.12.0,>=0.11 in /databricks/python3/lib/python3.7/site-packages (from pmdarima) (0.11.0)
Requirement already satisfied: pandas>=0.19 in /databricks/python3/lib/python3.7/site-packages (from pmdarima) (1.2.0)
Requirement already satisfied: numpy>=1.17.3 in /databricks/python3/lib/python3.7/site-packages (from pmdarima) (1.18.1)
Requirement already satisfied: scipy>=1.3.2 in /databricks/python3/lib/python3.7/site-packages (from pmdarima) (1.4.1)
Requirement already satisfied: scikit-learn>=0.22 in /databricks/python3/lib/python3.7/site-packages (from pmdarima) (0.22.1)
Requirement already satisfied: Cython<0.29.18,>=0.29 in /databricks/python3/lib/python3.7/site-packages (from pmdarima) (0.29.15)
Requirement already satisfied: setuptools!=50.0.0,>=38.6.0 in /usr/local/lib/python3.7/dist-packages (from pmdarima) (45.2.0)
Requirement already satisfied: patsy>=0.5 in /databricks/python3/lib/python3.7/site-packages (from statsmodels!=0.12.0,>=0.11->pmdarima) (0.5.1)
Requirement already satisfied: python-dateutil>=2.7.3 in /databricks/python3/lib/python3.7/site-packages (from pandas>=0.19->pmdarima) (2.8.1)
Requirement already satisfied: pytz>=2017.3 in /databricks/python3/lib/python3.7/site-packages (from pandas>=0.19->pmdarima) (2019.3)
Requirement already satisfied: six in /databricks/python3/lib/python3.7/site-packages (from patsy>=0.5->statsmodels!=0.12.0,>=0.11->pmdarima) (1.14.0)
 WARNING: You are using pip version 20.0.2; however, version 20.3.3 is available.
You should consider upgrading via the '/databricks/python3/bin/python3.7 -m pip install --upgrade pip' command.

In [82]:
#Arima Model Fitting

model1 = auto_arima(filtered_ts, trace=True, error_action='ignore', seasonal=True, suppress_warnings=True)
model1.fit(filtered_ts)

model2 = auto_arima(filtered_ts_log, trace=True, error_action='ignore',seasonal=True, suppress_warnings=True)
model2.fit(filtered_ts_log)


Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0] intercept : AIC=675.660, Time=0.29 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept : AIC=729.241, Time=0.01 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept : AIC=681.988, Time=0.08 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept : AIC=701.992, Time=0.07 sec
 ARIMA(0,0,0)(0,0,0)[0] : AIC=878.543, Time=0.01 sec
 ARIMA(1,0,2)(0,0,0)[0] intercept : AIC=678.186, Time=0.04 sec
 ARIMA(2,0,1)(0,0,0)[0] intercept : AIC=679.588, Time=0.16 sec
 ARIMA(3,0,2)(0,0,0)[0] intercept : AIC=666.422, Time=0.25 sec
 ARIMA(3,0,1)(0,0,0)[0] intercept : AIC=666.514, Time=0.24 sec
 ARIMA(4,0,2)(0,0,0)[0] intercept : AIC=667.898, Time=0.31 sec
 ARIMA(3,0,3)(0,0,0)[0] intercept : AIC=666.126, Time=0.32 sec
 ARIMA(2,0,3)(0,0,0)[0] intercept : AIC=inf, Time=0.29 sec
 ARIMA(4,0,3)(0,0,0)[0] intercept : AIC=667.178, Time=0.38 sec
 ARIMA(3,0,4)(0,0,0)[0] intercept : AIC=658.939, Time=0.37 sec
 ARIMA(2,0,4)(0,0,0)[0] intercept : AIC=669.381, Time=0.35 sec
 ARIMA(4,0,4)(0,0,0)[0] intercept : AIC=657.589, Time=0.44 sec
 ARIMA(5,0,4)(0,0,0)[0] intercept : AIC=661.125, Time=0.47 sec
 ARIMA(4,0,5)(0,0,0)[0] intercept : AIC=inf, Time=0.50 sec
 ARIMA(3,0,5)(0,0,0)[0] intercept : AIC=inf, Time=0.39 sec
 ARIMA(5,0,3)(0,0,0)[0] intercept : AIC=660.401, Time=0.41 sec
 ARIMA(5,0,5)(0,0,0)[0] intercept : AIC=inf, Time=0.56 sec
 ARIMA(4,0,4)(0,0,0)[0] : AIC=inf, Time=0.47 sec

Best model: ARIMA(4,0,4)(0,0,0)[0] intercept
Total fit time: 6.413 seconds
Performing stepwise search to minimize aic
 ARIMA(2,0,2)(0,0,0)[0] intercept : AIC=-58.162, Time=0.24 sec
 ARIMA(0,0,0)(0,0,0)[0] intercept : AIC=-3.025, Time=0.03 sec
 ARIMA(1,0,0)(0,0,0)[0] intercept : AIC=-48.402, Time=0.10 sec
 ARIMA(0,0,1)(0,0,0)[0] intercept : AIC=-30.321, Time=0.03 sec
 ARIMA(0,0,0)(0,0,0)[0] : AIC=333.788, Time=0.01 sec
 ARIMA(1,0,2)(0,0,0)[0] intercept : AIC=-54.870, Time=0.26 sec
 ARIMA(2,0,1)(0,0,0)[0] intercept : AIC=-55.804, Time=0.22 sec
 ARIMA(3,0,2)(0,0,0)[0] intercept : AIC=-58.827, Time=0.31 sec
 ARIMA(3,0,1)(0,0,0)[0] intercept : AIC=-62.975, Time=0.24 sec
 ARIMA(3,0,0)(0,0,0)[0] intercept : AIC=-56.912, Time=0.12 sec
 ARIMA(4,0,1)(0,0,0)[0] intercept : AIC=-59.531, Time=0.30 sec
 ARIMA(2,0,0)(0,0,0)[0] intercept : AIC=-53.646, Time=0.13 sec
 ARIMA(4,0,0)(0,0,0)[0] intercept : AIC=-54.913, Time=0.39 sec
 ARIMA(4,0,2)(0,0,0)[0] intercept : AIC=-56.142, Time=0.44 sec
 ARIMA(3,0,1)(0,0,0)[0] : AIC=-47.442, Time=0.17 sec

Best model: ARIMA(3,0,1)(0,0,0)[0] intercept
Total fit time: 2.995 seconds
Out[24]: ARIMA(maxiter=50, method='lbfgs', order=(3, 0, 1), out_of_sample_size=0,
 scoring='mse', scoring_args={}, seasonal_order=(0, 0, 0, 0),
 start_params=None, suppress_warnings=True, trend=None,
 with_intercept=True)

Out[25]: array([2175.18984619, 1439.8601396 , 1552.91915872, 2137.63038277,
 2373.33383278, 2387.61908524, 2542.72759123, 2462.28218047,
 2154.30612425, 1882.80682654, 1762.60479975, 1726.33431221,
 1513.06511402, 1661.59807946, 1842.00263381, 2365.28961787,
 2450.86579317, 2615.72850374, 2658.40560647, 2631.21347952,
 2364.37605043, 1977.89937452, 2005.67786354, 1967.23157508,
 1556.58159674, 1656.81351132, 1887.79514195, 2202.10939547,
 2569.58153695, 2660.79708615, 2758.79938155, 2681.26552511,
 2615.4465985 , 2031.62125261, 2355.86811362, 1760.45896626,
 1692.20738234, 1717.86253591, 1923.44821827, 2369.46582234,
 2576.32282865, 2859.46960429, 2981.46022115, 2900.50710796,
 2700.41526121, 2147.5829399 , 2352.82994082, 1968.19227748])

In [84]:
# Arima Forecast Generation

#from plotly.offline import plot
#import matplotlib.pyplot as plt

forecast1 = model1.predict(n_periods=12)
#plt.plot(forecast1)
filtered_ts['prediction']=model1.predict_in_sample(filtered_ts)
filtered_test_ts['prediction']=forecast1

future_forecast1 = pd.DataFrame(filtered_test_ts,index = filtered_test_ts.index,columns=["prediction"])
future_forecast1['prediction']=forecast1
fcst_plot=pd.concat([filtered_ts,filtered_test_ts], axis=0)
#sns.lineplot(data=fcst_plot, x="month_year", y="y")
#fcst_plot.plot()
#plt.show()
fcst_plot['month_year']=fcst_plot.index
display(fcst_plot)

y,prediction,month_year
1247,2175.1898461861015,2013-01-01T00:00:00.000+0000
1283,1439.860139601839,2013-02-01T00:00:00.000+0000
1804,1552.9191587248615,2013-03-01T00:00:00.000+0000
2064,2137.6303827707916,2013-04-01T00:00:00.000+0000
2110,2373.3338327781294,2013-05-01T00:00:00.000+0000
2262,2387.6190852357777,2013-06-01T00:00:00.000+0000
2396,2542.7275912332652,2013-07-01T00:00:00.000+0000
2179,2462.2821804717787,2013-08-01T00:00:00.000+0000
1972,2154.306124251589,2013-09-01T00:00:00.000+0000
1826,1882.806826542846,2013-10-01T00:00:00.000+0000


In [85]:
# import matplotlib.pyplot as plt
# ax = plt.gca()

# fcst_plot.plot(kind='line',x='index',y='y',ax=ax)
# df.plot(kind='line',x='index',y='prediction', color='red', ax=ax)

--------------------------------------------------------------------------- 
 ImportError Traceback (most recent call last)
 <command-4300904910114480> in <module> 
 2 ax = plt . gca ( ) 
 3 
 ----> 4 fcst_plot . plot ( kind = 'line' , x = 'index' , y = 'y' , ax = ax ) 
 5 df . plot ( kind = 'line' , x = 'index' , y = 'prediction' , color = 'red' , ax = ax ) 

 /databricks/python/lib/python3.7/site-packages/pandas/plotting/_core.py in __call__ (self, *args, **kwargs) 
 765 _dataframe_kinds = ( "scatter" , "hexbin" ) 
 766 _kind_aliases = { "density" : "kde" } 
 --> 767 _all_kinds = _common_kinds + _series_kinds + _dataframe_kinds
 768 
 769 def __init__ ( self , data ) : 

 /databricks/python/lib/python3.7/site-packages/pandas/plotting/_core.py in _get_plot_backend (backend) 
 1667 the matplotlib function that is used under the hood . 
 1668 
 -> 1669 Examples
 1670 - - - - - - - - 
 1671 The following examples are generated with random data from 

 ImportError : matplotlib is required for plotting when the default backend "matplotlib" is selected.

In [86]:
# Plot with log
forecast2 = np.exp(model2.predict(n_periods=12))
plt.plot(forecast2)
forecast2


Out[104]:

array([1755.33935658, 1615.23316729, 1777.46421763, 1848.35010466,
 2117.31599789, 2157.24521062, 2328.00043543, 2225.20430767,
 2276.55479588, 2118.89672097, 2154.71652995, 2033.4099416 ])

In [87]:
import numpy as np
filtered_test_ts['y_pred']=forecast1
y_actual=filtered_test_ts['y']
y_pred=filtered_test_ts['y_pred']
MAPE=np.mean(np.abs((y_actual - y_pred) / y_actual)) * 100
print("MAPE without log:", MAPE)

smape1=pm.metrics.smape(filtered_test_ts['y'],forecast1)
print("SMAPE without log:",smape1)



filtered_test_ts['y_pred']=forecast2
y_actual=filtered_test_ts['y']
#y_pred=filtered_test_ts['y_pred']
MAPE2=np.mean(np.abs((y_actual - forecast2) / y_actual)) * 100
print("MAPE with log:",MAPE2)

smape2=pm.metrics.smape(filtered_test_ts['y'],forecast2)
print("SMAPE with log:", smape2)



MAPE without log: 12.822049930694199
SMAPE without log: 14.068956644193664
MAPE with log: 17.997898113906636
SMAPE with log: 20.480815059405195

In [88]:
model.summary()

In [89]:
# Holtwinter Implementation

from statsmodels.tsa.holtwinters import ExponentialSmoothing
#import pmdarima as pm
import numpy as np

traindf.head()
traindf_ts=traindf[['month_year','y']]
traindf_ts=traindf_ts.groupby("month_year").sum()

holt_model = ExponentialSmoothing(traindf_ts, trend="add",seasonal="add", seasonal_periods=12)
holt_fit=holt_model.fit()
pred=holt_fit.forecast(12)

testdf_ts=testdf[['month_year','y']]
testdf_ts=testdf_ts.groupby("month_year").sum()
testdf_ts['pred']=pred
smape1=pm.metrics.smape(testdf_ts['y'],pred)



traindf_ts_log=traindf_ts
traindf_ts_log['y']=np.log(traindf_ts['y'])
holt_model = ExponentialSmoothing(traindf_ts, trend="add",seasonal="add", seasonal_periods=12)
holt_fit=holt_model.fit()
pred2=np.exp(holt_fit.forecast(12))
smape2=pm.metrics.smape(testdf_ts['y'],pred2)


print("SMAPE without log:",smape1)
print("SMAPE with log:",smape2)



--------------------------------------------------------------------------- 
 NameError Traceback (most recent call last)
 <command-3401161627101954> in <module> 
 16 testdf_ts = testdf_ts . groupby ( "month_year" ) . sum ( ) 
 17 testdf_ts [ 'pred' ] = pred
 ---> 18 smape1 = pm . metrics . smape ( testdf_ts [ 'y' ] , pred ) 
 19 
 20 

 NameError : name 'pm' is not defined

In [90]:
!pip install tbats

Collecting tbats
 Downloading tbats-1.1.0-py3-none-any.whl (43 kB)
███████▌ | 10 kB 27.9 MB/s eta 0:00:01 |███████████████ | 20 kB 18.6 MB/s eta 0:00:01 |██████████████████████▍ | 30 kB 15.5 MB/s eta 0:00:01 |██████████████████████████████ | 40 kB 11.4 MB/s eta 0:00:01 |████████████████████████████████| 43 kB 1.6 MB/s 
Collecting pmdarima
 Downloading pmdarima-1.8.0-cp37-cp37m-manylinux1_x86_64.whl (1.5 MB)
▎ | 10 kB 29.1 MB/s eta 0:00:01 |▌ | 20 kB 32.2 MB/s eta 0:00:01 |▊ | 30 kB 36.1 MB/s eta 0:00:01 |█ | 40 kB 21.5 MB/s eta 0:00:01 |█▏ | 51 kB 22.9 MB/s eta 0:00:01 |█▍ | 61 kB 23.9 MB/s eta 0:00:01 |█▋ | 71 kB 21.1 MB/s eta 0:00:01 |█▉ | 81 kB 22.8 MB/s eta 0:00:01 |██ | 92 kB 24.3 MB/s eta 0:00:01 |██▎ | 102 kB 21.0 MB/s eta 0:00:01 |██▌ | 112 kB 21.0 MB/s eta 0:00:01 |██▊ | 122 kB 21.0 MB/s eta 0:00:01 |███ | 133 kB 21.0 MB/s eta 0:00:01 |███▏ | 143 kB 21.0 MB/s eta 0:00:01 |███▍ | 153 kB 21.0 MB/s eta 0:00:01 |███▋ | 163 kB 21.0 MB/s eta 0:00:01 |███▉ | 174 kB 21.0 MB/s eta 0:00:01 |████ | 184 kB 21.0 MB/s eta 0:00:01 |████▎ | 194 kB 21.0 MB/s eta 0:00:01 |████▌ | 204 kB 21.0 MB/s eta 0:00:01 |████▊ | 215 kB 21.0 MB/s eta 0:00:01 |█████ | 225 kB 21.0 MB/s eta 0:00:01 |█████▏ | 235 kB 21.0 MB/s eta 0:00:01 |█████▍ | 245 kB 21.0 MB/s eta 0:00:01 |█████▋ | 256 kB 21.0 MB/s eta 0:00:01 |█████▉ | 266 kB 21.0 MB/s eta 0:00:01 |██████ | 276 kB 21.0 MB/s eta 0:00:01 |██████▎ | 286 kB 21.0 MB/s eta 0:00:01 |██████▌ | 296 kB 21.0 MB/s eta 0:00:01 |██████▊ | 307 kB 21.0 MB/s eta 0:00:01 |███████ | 317 kB 21.0 MB/s eta 0:00:01 |███████▏ | 327 kB 21.0 MB/s eta 0:00:01 |███████▍ | 337 kB 21.0 MB/s eta 0:00:01 |███████▋ | 348 kB 21.0 MB/s eta 0:00:01 |███████▉ | 358 kB 21.0 MB/s eta 0:00:01 |████████ | 368 kB 21.0 MB/s eta 0:00:01 |████████▎ | 378 kB 21.0 MB/s eta 0:00:01 |████████▌ | 389 kB 21.0 MB/s eta 0:00:01 |████████▊ | 399 kB 21.0 MB/s eta 0:00:01 |█████████ | 409 kB 21.0 MB/s eta 0:00:01 |█████████▏ | 419 kB 21.0 MB/s eta 0:00:01 |█████████▍ | 430 kB 21.0 MB/s eta 0:00:01 |█████████▋ | 440 kB 21.0 MB/s eta 0:00:01 |█████████▉ | 450 kB 21.0 MB/s eta 0:00:01 |██████████ | 460 kB 21.0 MB/s eta 0:00:01 |██████████▎ | 471 kB 21.0 MB/s eta 0:00:01 |██████████▌ | 481 kB 21.0 MB/s eta 0:00:01 |██████████▊ | 491 kB 21.0 MB/s eta 0:00:01 |███████████ | 501 kB 21.0 MB/s eta 0:00:01 |███████████▏ | 512 kB 21.0 MB/s eta 0:00:01 |███████████▌ | 522 kB 21.0 MB/s eta 0:00:01 |███████████▊ | 532 kB 21.0 MB/s eta 0:00:01 |████████████ | 542 kB 21.0 MB/s eta 0:00:01 |████████████▏ | 552 kB 21.0 MB/s eta 0:00:01 |████████████▍ | 563 kB 21.0 MB/s eta 0:00:01 |████████████▋ | 573 kB 21.0 MB/s eta 0:00:01 |████████████▉ | 583 kB 21.0 MB/s eta 0:00:01 |█████████████ | 593 kB 21.0 MB/s eta 0:00:01 |█████████████▎ | 604 kB 21.0 MB/s eta 0:00:01 |█████████████▌ | 614 kB 21.0 MB/s eta 0:00:01 |█████████████▊ | 624 kB 21.0 MB/s eta 0:00:01 |██████████████ | 634 kB 21.0 MB/s eta 0:00:01 |██████████████▏ | 645 kB 21.0 MB/s eta 0:00:01 |██████████████▍ | 655 kB 21.0 MB/s eta 0:00:01 |██████████████▋ | 665 kB 21.0 MB/s eta 0:00:01 |██████████████▉ | 675 kB 21.0 MB/s eta 0:00:01 |███████████████ | 686 kB 21.0 MB/s eta 0:00:01 |███████████████▎ | 696 kB 21.0 MB/s eta 0:00:01 |███████████████▌ | 706 kB 21.0 MB/s eta 0:00:01 |███████████████▊ | 716 kB 21.0 MB/s eta 0:00:01 |████████████████ | 727 kB 21.0 MB/s eta 0:00:01 |████████████████▏ | 737 kB 21.0 MB/s eta 0:00:01 |████████████████▍ | 747 kB 21.0 MB/s eta 0:00:01 |████████████████▋ | 757 kB 21.0 MB/s eta 0:00:01 |████████████████▉ | 768 kB 21.0 MB/s eta 0:00:01 |█████████████████ | 778 kB 21.0 MB/s eta 0:00:01 |█████████████████▎ | 788 kB 21.0 MB/s eta 0:00:01 |█████████████████▌ | 798 kB 21.0 MB/s eta 0:00:01 |█████████████████▊ | 808 kB 21.0 MB/s eta 0:00:01 |██████████████████ | 819 kB 21.0 MB/s eta 0:00:01 |██████████████████▏ | 829 kB 21.0 MB/s eta 0:00:01 |██████████████████▍ | 839 kB 21.0 MB/s eta 0:00:01 |████

In [91]:
traindf.head()

Out[7]:

,ds,store,item,y,year,month,weekday,month_year
911174,2013-01-01,10,50,33,2013,1,1,2013-01-01
911175,2013-01-02,10,50,37,2013,1,2,2013-01-01
911176,2013-01-03,10,50,46,2013,1,3,2013-01-01
911177,2013-01-04,10,50,51,2013,1,4,2013-01-01
911178,2013-01-05,10,50,41,2013,1,5,2013-01-01


In [92]:
# Tbats Implementation

from tbats import TBATS

estimator1 = TBATS(seasonal_periods=[12,24])
model1 = estimator1.fit(traindf_ts)
forecast1=model1.forecast(12)

#model.predict_in_sample(traindf_ts)
testdf_ts['pred1'] = forecast1


estimator2 = TBATS()
model2 = estimator2.fit(traindf_ts)
forecast2=model2.forecast(12)

#model.predict_in_sample(traindf_ts)
#testdf_ts['pred2'] = forecast2

mape1=np.mean(np.abs((testdf_ts['y'] - forecast1) / testdf_ts['y'])) * 100
mape2=np.mean(np.abs((testdf_ts['y'] - forecast2) / testdf_ts['y'])) * 100

tbats_train_df=traindf_ts.copy()
tbats_train_df['pred1']=np.nan

tbats_test_df=testdf_ts.copy()
tbats_test_df['pred1']=forecast1

tbatsdf=pd.concat([tbats_train_df, tbats_test_df], axis=0)


tbatsdf['month_year']=filtered['month_year'].unique()
display(tbatsdf)

#print("MAPE1", mape1)
#print("MAPE2", mape2)

y,pred1,month_year
1247,null,2013-01-01T00:00:00.000+0000
1283,null,2013-02-01T00:00:00.000+0000
1804,null,2013-03-01T00:00:00.000+0000
2064,null,2013-04-01T00:00:00.000+0000
2110,null,2013-05-01T00:00:00.000+0000
2262,null,2013-06-01T00:00:00.000+0000
2396,null,2013-07-01T00:00:00.000+0000
2179,null,2013-08-01T00:00:00.000+0000
1972,null,2013-09-01T00:00:00.000+0000
1826,null,2013-10-01T00:00:00.000+0000


In [93]:
#Tbats Evaluation

print("MAPE1 with Seasonal Periods", mape1)
print("MAPE2 with default ", mape2)

MAPE1 with Seasonal Periods 3.8235801684955097
MAPE2 with default 20.880106911809484

In [94]:
!pip install xgboost

Collecting xgboost
 Downloading xgboost-1.3.1-py3-none-manylinux2010_x86_64.whl (157.5 MB)
 | 10 kB 25.5 MB/s eta 0:00:07 | | 20 kB 18.3 MB/s eta 0:00:09 | | 30 kB 10.4 MB/s eta 0:00:16 | | 40 kB 5.6 MB/s eta 0:00:28 | | 51 kB 6.8 MB/s eta 0:00:24 | | 61 kB 7.5 MB/s eta 0:00:22 | | 71 kB 7.9 MB/s eta 0:00:20 | | 81 kB 8.0 MB/s eta 0:00:20 | | 92 kB 8.4 MB/s eta 0:00:19 | | 102 kB 7.1 MB/s eta 0:00:23 | | 112 kB 7.1 MB/s eta 0:00:23 | | 122 kB 7.1 MB/s eta 0:00:23 | | 133 kB 7.1 MB/s eta 0:00:23 | | 143 kB 7.1 MB/s eta 0:00:23 | | 153 kB 7.1 MB/s eta 0:00:23 | | 163 kB 7.1 MB/s eta 0:00:23 | | 174 kB 7.1 MB/s eta 0:00:23 | | 184 kB 7.1 MB/s eta 0:00:23 | | 194 kB 7.1 MB/s eta 0:00:23 | | 204 kB 7.1 MB/s eta 0:00:23 | | 215 kB 7.1 MB/s eta 0:00:23 | | 225 kB 7.1 MB/s eta 0:00:23 | | 235 kB 7.1 MB/s eta 0:00:23 | | 245 kB 7.1 MB/s eta 0:00:23 | | 256 kB 7.1 MB/s eta 0:00:23 | | 266 kB 7.1 MB/s eta 0:00:23 | | 276 kB 7.1 MB/s eta 0:00:23 | | 286 kB 7.1 MB/s eta 0:00:23 | | 296 kB 7.1 MB/s eta 0:00:23 | | 307 kB 7.1 MB/s eta 0:00:23 | | 317 kB 7.1 MB/s eta 0:00:23 | | 327 kB 7.1 MB/s eta 0:00:23 | | 337 kB 7.1 MB/s eta 0:00:23 | | 348 kB 7.1 MB/s eta 0:00:23 | | 358 kB 7.1 MB/s eta 0:00:23 | | 368 kB 7.1 MB/s eta 0:00:23 | | 378 kB 7.1 MB/s eta 0:00:23 | | 389 kB 7.1 MB/s eta 0:00:23 | | 399 kB 7.1 MB/s eta 0:00:23 | | 409 kB 7.1 MB/s eta 0:00:23 | | 419 kB 7.1 MB/s eta 0:00:23 | | 430 kB 7.1 MB/s eta 0:00:23 | | 440 kB 7.1 MB/s eta 0:00:23 | | 450 kB 7.1 MB/s eta 0:00:23 | | 460 kB 7.1 MB/s eta 0:00:23 | | 471 kB 7.1 MB/s eta 0:00:23 | | 481 kB 7.1 MB/s eta 0:00:23 | | 491 kB 7.1 MB/s eta 0:00:23 | | 501 kB 7.1 MB/s eta 0:00:23 | | 512 kB 7.1 MB/s eta 0:00:23 | | 522 kB 7.1 MB/s eta 0:00:23 | | 532 kB 7.1 MB/s eta 0:00:23 | | 542 kB 7.1 MB/s eta 0:00:23 |▏ | 552 kB 7.1 MB/s eta 0:00:23 |▏ | 563 kB 7.1 MB/s eta 0:00:23 |▏ | 573 kB 7.1 MB/s eta 0:00:23 |▏ | 583 kB 7.1 MB/s eta 0:00:23 |▏ | 593 kB 7.1 MB/s eta 0:00:23 |▏ | 604 kB 7.1 MB/s eta 0:00:23 |▏ | 614 kB 7.1 MB/s eta 0:00:23 |▏ | 624 kB 7.1 MB/s eta 0:00:23 |▏ | 634 kB 7.1 MB/s eta 0:00:23 |▏ | 645 kB 7.1 MB/s eta 0:00:23 |▏ | 655 kB 7.1 MB/s eta 0:00:23 |▏ | 665 kB 7.1 MB/s eta 0:00:23 |▏ | 675 kB 7.1 MB/s eta 0:00:23 |▏ | 686 kB 7.1 MB/s eta 0:00:23 |▏ | 696 kB 7.1 MB/s eta 0:00:23 |▏ | 706 kB 7.1 MB/s eta 0:00:23 |▏ | 716 kB 7.1 MB/s eta 0:00:23 |▏ | 727 kB 7.1 MB/s eta 0:00:23 |▏ | 737 kB 7.1 MB/s eta 0:00:23 |▏ | 747 kB 7.1 MB/s eta 0:00:23 |▏ | 757 kB 7.1 MB/s eta 0:00:23 |▏ | 768 kB 7.1 MB/s eta 0:00:23 |▏ | 778 kB 7.1 MB/s eta 0:00:23 |▏ | 788 kB 7.1 MB/s eta 0:00:23 |▏ | 798 kB 7.1 MB/s eta 0:00:23 |▏ | 808 kB 7.1 MB/s eta 0:00:23 |▏ | 819 kB 7.1 MB/s eta 0:00:23 |▏ | 829 kB 7.1 MB/s eta 0:00:23 |▏ | 839 kB 7.1 MB/s eta 0:00:23 |▏ | 849 kB 7.1 MB/s eta 0:00:22 |▏ | 860 kB 7.1 MB/s eta 0:00:22 |▏ | 870 kB 7.1 MB/s eta 0:00:22 |▏ | 880 kB 7.1 MB/s eta 0:00:22 |▏ | 890 kB 7.1 MB/s eta 0:00:22 |▏ | 901 kB 7.1 MB/s eta 0:00:22 |▏ | 911 kB 7.1 MB/s eta 0:00:22 |▏ | 921 kB 7.1 MB/s eta 0:00:22 |▏ | 931 kB 7.1 MB/s eta 0:00:22 |▏ | 942 kB 7.1 MB/s eta 0:00:22 |▏ | 952 kB 7.1 MB/s eta 0:00:22 |▏ | 962 kB 7.1 MB/s eta 0:00:22 |▏ | 972 kB 7.1 MB/s eta 0:00:22 |▏ | 983 kB 7.1 MB/s eta 0:00:22 |▏ | 993 kB 7.1 MB/s eta 0:00:22 |▏ | 1.0 MB 7.1 MB/s eta 0:00:22 |▏ | 1.0 MB 7.1 MB/s eta 0:00:22 |▏ | 1.0 MB 7.1 MB/s eta 0:00:22 |▏ | 1.0 MB 7.1 MB/s eta 0:00:22 |▏ | 1.0 MB 7.1 MB/s eta 0:00:22 |▏ | 1.1 MB 7.1 MB/s eta 0:00:22 |▏ | 1.1 MB 7.1 MB/s eta 0:00:22 |▏ | 1.1 MB 7.1 MB/s eta 0:00:22 |▏ | 1.1 MB 7.1 MB/s eta 0:00:22 |▎ | 1.1 MB 7.1 MB/s eta 0:00:22 |▎ | 1.1 MB 7.1 MB/s eta 0:00:22 |▎ | 1.1 MB 7.1 MB/s eta 0:00:22 |▎ | 1.1 MB 7.1 MB/s eta 0:00:22 |▎ | 1.1 MB 7.1 MB/s eta 0:00:22 |▎ | 1.1 MB 7.1 MB/s eta 0:00:22 |▎ | 1.2 MB 7.1 MB/s eta 0:00:22 |▎ | 1.2 MB 7.1 MB/s eta 0:00:22 |▎ | 1.2 MB 7.1 MB/s eta 0:00:22 |▎ | 1.2 MB 7.1 MB/s eta 0:00:22 |▎ | 1.2 MB 7.1 MB/s et

Out[18]: 911174 1
911175 2
911176 3
911177 4
911178 5
 ..
912995 27
912996 28
912997 29
912998 30
912999 31
Name: ds, Length: 1826, dtype: int64

In [96]:
#XGBoost Implementation

import xgboost as xgb
import numpy as np
xgbdf_train=filtered[filtered['year']<2017]
xgbdf_test=filtered[filtered['year']>=2017]

x_train=xgbdf_train[['month', 'weekday', 'quarter', 'year', 'dayofmonth', 'dayofyear']]
y_train=xgbdf_train['y']

x_test=xgbdf_test[['month', 'weekday', 'quarter', 'year',  'dayofmonth', 'dayofyear']]
y_test=xgbdf_test['y']

reg = xgb.XGBRegressor(n_estimators=1000)
reg.fit(x_train, y_train,
        eval_set=[(x_train, y_train), (x_test, y_test)],
        early_stopping_rounds=5, #stop if 50 consequent rounds without decrease of error
        verbose=False) # Change verbose to True if you want to see it train


y_test_pred = reg.predict(x_test)
y_train_pred= reg.predict(x_train)
y_train.index=filtered[filtered['year']<2017].ds

df_xgb = pd.DataFrame({'date' : []})
df_xgb['date']=filtered.ds
df_xgb['y']=filtered['y']
df_xgb['y_pred']=np.concatenate([y_train_pred, y_test_pred])
df_xgb['month_year']=filtered['month_year']
display(df_xgb)

date,y,y_pred,month_year
2013-01-01,33,36.509182,2013-01-01T00:00:00.000+0000
2013-01-02,37,41.735516,2013-01-01T00:00:00.000+0000
2013-01-03,46,43.50461,2013-01-01T00:00:00.000+0000
2013-01-04,51,45.01748,2013-01-01T00:00:00.000+0000
2013-01-05,41,43.70672,2013-01-01T00:00:00.000+0000
2013-01-06,41,45.178974,2013-01-01T00:00:00.000+0000
2013-01-07,38,36.997517,2013-01-01T00:00:00.000+0000
2013-01-08,33,34.464294,2013-01-01T00:00:00.000+0000
2013-01-09,35,36.90102,2013-01-01T00:00:00.000+0000
2013-01-10,41,37.525616,2013-01-01T00:00:00.000+0000


In [97]:
# XGBoost Evaluation

df_xgb_monthly=df_xgb.groupby("month_year", as_index=False).sum()
display(df_xgb_monthly)
print("MAPE:", np.mean(np.abs((xgbdf_test['y'] - y_test_pred) / xgbdf_test['y'])) * 100)


month_year,y,y_pred
2013-01-01T00:00:00.000+0000,1247,1258.6807
2013-02-01T00:00:00.000+0000,1283,1295.6426
2013-03-01T00:00:00.000+0000,1804,1803.759
2013-04-01T00:00:00.000+0000,2064,2022.7223
2013-05-01T00:00:00.000+0000,2110,2127.4548
2013-06-01T00:00:00.000+0000,2262,2266.088
2013-07-01T00:00:00.000+0000,2396,2367.0632
2013-08-01T00:00:00.000+0000,2179,2185.572
2013-09-01T00:00:00.000+0000,1972,1985.1068
2013-10-01T00:00:00.000+0000,1826,1818.6943


MAPE: 9.75886184899004

In [98]:
!pip install keras
!pip install tensorflow

Requirement already satisfied: keras in /databricks/python3/lib/python3.7/site-packages (2.4.3)
Requirement already satisfied: pyyaml in /databricks/python3/lib/python3.7/site-packages (from keras) (5.3.1)
Requirement already satisfied: h5py in /databricks/python3/lib/python3.7/site-packages (from keras) (3.1.0)
Requirement already satisfied: scipy>=0.14 in /databricks/python3/lib/python3.7/site-packages (from keras) (1.4.1)
Requirement already satisfied: numpy>=1.9.1 in /databricks/python3/lib/python3.7/site-packages (from keras) (1.18.1)
Requirement already satisfied: cached-property; python_version < "3.8" in /databricks/python3/lib/python3.7/site-packages (from h5py->keras) (1.5.2)
 WARNING: You are using pip version 20.0.2; however, version 20.3.3 is available.
You should consider upgrading via the '/databricks/python3/bin/python3.7 -m pip install --upgrade pip' command. 
Collecting tensorflow
 Downloading tensorflow-2.4.0-cp37-cp37m-manylinux2010_x86_64.whl (394.7 MB)
 | 10 kB 25.0 MB/s eta 0:00:16 | | 20 kB 18.2 MB/s eta 0:00:22 | | 30 kB 8.8 MB/s eta 0:00:45 | | 40 kB 6.3 MB/s eta 0:01:03 | | 51 kB 7.5 MB/s eta 0:00:53 | | 61 kB 8.7 MB/s eta 0:00:46 | | 71 kB 8.7 MB/s eta 0:00:46 | | 81 kB 9.0 MB/s eta 0:00:44 | | 92 kB 8.4 MB/s eta 0:00:48 | | 102 kB 7.5 MB/s eta 0:00:53 | | 112 kB 7.5 MB/s eta 0:00:53 | | 122 kB 7.5 MB/s eta 0:00:53 | | 133 kB 7.5 MB/s eta 0:00:53 | | 143 kB 7.5 MB/s eta 0:00:53 | | 153 kB 7.5 MB/s eta 0:00:53 | | 163 kB 7.5 MB/s eta 0:00:53 | | 174 kB 7.5 MB/s eta 0:00:53 | | 184 kB 7.5 MB/s eta 0:00:53 | | 194 kB 7.5 MB/s eta 0:00:53 | | 204 kB 7.5 MB/s eta 0:00:53 | | 215 kB 7.5 MB/s eta 0:00:53 | | 225 kB 7.5 MB/s eta 0:00:53 | | 235 kB 7.5 MB/s eta 0:00:53 | | 245 kB 7.5 MB/s eta 0:00:53 | | 256 kB 7.5 MB/s eta 0:00:53 | | 266 kB 7.5 MB/s eta 0:00:53 | | 276 kB 7.5 MB/s eta 0:00:53 | | 286 kB 7.5 MB/s eta 0:00:53 | | 296 kB 7.5 MB/s eta 0:00:53 | | 307 kB 7.5 MB/s eta 0:00:53 | | 317 kB 7.5 MB/s eta 0:00:53 | | 327 kB 7.5 MB/s eta 0:00:53 | | 337 kB 7.5 MB/s eta 0:00:53 | | 348 kB 7.5 MB/s eta 0:00:53 | | 358 kB 7.5 MB/s eta 0:00:53 | | 368 kB 7.5 MB/s eta 0:00:53 | | 378 kB 7.5 MB/s eta 0:00:53 | | 389 kB 7.5 MB/s eta 0:00:53 | | 399 kB 7.5 MB/s eta 0:00:53 | | 409 kB 7.5 MB/s eta 0:00:53 | | 419 kB 7.5 MB/s eta 0:00:53 | | 430 kB 7.5 MB/s eta 0:00:53 | | 440 kB 7.5 MB/s eta 0:00:53 | | 450 kB 7.5 MB/s eta 0:00:53 | | 460 kB 7.5 MB/s eta 0:00:53 | | 471 kB 7.5 MB/s eta 0:00:53 | | 481 kB 7.5 MB/s eta 0:00:53 | | 491 kB 7.5 MB/s eta 0:00:53 | | 501 kB 7.5 MB/s eta 0:00:53 | | 512 kB 7.5 MB/s eta 0:00:53 | | 522 kB 7.5 MB/s eta 0:00:53 | | 532 kB 7.5 MB/s eta 0:00:53 | | 542 kB 7.5 MB/s eta 0:00:53 | | 552 kB 7.5 MB/s eta 0:00:53 | | 563 kB 7.5 MB/s eta 0:00:53 | | 573 kB 7.5 MB/s eta 0:00:53 | | 583 kB 7.5 MB/s eta 0:00:53 | | 593 kB 7.5 MB/s eta 0:00:53 | | 604 kB 7.5 MB/s eta 0:00:53 | | 614 kB 7.5 MB/s eta 0:00:53 | | 624 kB 7.5 MB/s eta 0:00:53 | | 634 kB 7.5 MB/s eta 0:00:53 | | 645 kB 7.5 MB/s eta 0:00:53 | | 655 kB 7.5 MB/s eta 0:00:53 | | 665 kB 7.5 MB/s eta 0:00:53 | | 675 kB 7.5 MB/s eta 0:00:53 | | 686 kB 7.5 MB/s eta 0:00:53 | | 696 kB 7.5 MB/s eta 0:00:53 | | 706 kB 7.5 MB/s eta 0:00:53 | | 716 kB 7.5 MB/s eta 0:00:53 | | 727 kB 7.5 MB/s eta 0:00:53 | | 737 kB 7.5 MB/s eta 0:00:53 | | 747 kB 7.5 MB/s eta 0:00:53 | | 757 kB 7.5 MB/s eta 0:00:53 | | 768 kB 7.5 MB/s eta 0:00:53 | | 778 kB 7.5 MB/s eta 0:00:53 | | 788 kB 7.5 MB/s eta 0:00:53 | | 798 kB 7.5 MB/s eta 0:00:53 | | 808 kB 7.5 MB/s eta 0:00:53 | | 819 kB 7.5 MB/s eta 0:00:53 | | 829 kB 7.5 MB/s eta 0:00:53 | | 839 kB 7.5 MB/s eta 0:00:53 | | 849 kB 7.5 MB/s eta 0:00:53 | | 860 kB 7.5 MB/s eta 0:00:53 | | 870 kB 7.5 MB/s eta 0:00:53 | | 880 kB 7.5 MB/s eta 0:00:53 | | 890 kB 7.5 MB/s eta 0:00:53 | | 901 kB 7.5 MB/s eta 0:00:53 | | 911 kB 7.5 MB/s eta 0:00:53 | | 921 kB 7.5 MB/s eta 0:00:53 | | 931 kB 7.5 MB/s eta 0:00:53 | | 94

In [99]:
# libraries for LSTM implementation
from keras.models import Sequential
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.layers import Dense, LSTM



In [100]:
from sklearn.preprocessing import MinMaxScaler


lstmdf_train=filtered[filtered['year']<2017]
lstmdf_test=filtered[filtered['year']>=2017]

scaler=MinMaxScaler()

lstmdf_train=lstmdf_train[['ds', 'y']]
lstmdf_train.set_index(['ds'], inplace=True)
lstmdf_test=lstmdf_test[['ds', 'y']]
lstmdf_test.set_index(['ds'], inplace=True)

scaler.fit(lstmdf_train)

scaled_train=scaler.transform(lstmdf_train)
scaled_test=scaler.transform(lstmdf_test)

train_data_gen = TimeseriesGenerator(scaled_train,scaled_train,length=120, sampling_rate=1,stride=1,batch_size=3)
train_data_gen


Out[30]: <tensorflow.python.keras.preprocessing.sequence.TimeseriesGenerator at 0x7fa508f75ad0>